In [10]:
# -*- coding: utf-8 -*-
import argparse

"""
Created on Mon Oct 25 14:07:29 2021

@author: chumache
"""
import os
import json
import numpy as np
import torch
from torch import nn, optim
from torch.optim import lr_scheduler

from opts import parse_opts
from model import generate_model
import transforms 
from dataset import get_training_set, get_validation_set, get_test_set
from utils import adjust_learning_rate, save_checkpoint
from train import train_epoch
from validation import val_epoch
import time
import logging
from torch.autograd import Variable
import time
from utils import AverageMeter, calculate_accuracy
def parse_opts():
    parser = argparse.ArgumentParser()
    parser.add_argument('--phase', default='train', type=str) 
    parser.add_argument('--certification_method', default='MMCert', type=str) 
    parser.add_argument('--k1', default='4', type=int) 
    parser.add_argument('--k2', default='4000', type=int) 
    parser.add_argument('--k', default='3000', type=int) 
    parser.add_argument('--annotation_path', default='ravdess_preprocessing/annotations.txt', type=str, help='Annotation file path')
    parser.add_argument('--result_path', default='results', type=str, help='Result directory path')
    parser.add_argument('--store_name', default='model', type=str, help='Name to store checkpoints')
    parser.add_argument('--dataset', default='RAVDESS', type=str, help='Used dataset. Currently supporting Ravdess')
    parser.add_argument('--n_classes', default=8, type=int, help='Number of classes')
    
    parser.add_argument('--model', default='multimodalcnn', type=str, help='')
    parser.add_argument('--num_heads', default=1, type=int, help='number of heads, in the paper 1 or 4')
    
    parser.add_argument('--device', default='cuda', type=str, help='Specify the device to run. Defaults to cuda, fallsback to cpu')
    
    
    parser.add_argument('--sample_size', default=224, type=int, help='Video dimensions: ravdess = 224 ')
    parser.add_argument('--sample_duration', default=15, type=int, help='Temporal duration of inputs, ravdess = 15')
    
    parser.add_argument('--learning_rate', default=0.04, type=float, help='Initial learning rate (divided by 10 while training by lr scheduler)')
    parser.add_argument('--momentum', default=0.9, type=float, help='Momentum')
    parser.add_argument('--lr_steps', default=[40, 55, 65, 70, 200, 250], type=float, nargs="+", metavar='LRSteps', help='epochs to decay learning rate by 10')
    parser.add_argument('--dampening', default=0.9, type=float, help='dampening of SGD')
    parser.add_argument('--weight_decay', default=1e-3, type=float, help='Weight Decay')
    parser.add_argument('--lr_patience', default=10, type=int, help='Patience of LR scheduler. See documentation of ReduceLROnPlateau.')
    parser.add_argument('--batch_size', default=8, type=int, help='Batch Size')
    parser.add_argument('--n_epochs', default=100, type=int, help='Number of total epochs to run')
    
    parser.add_argument('--begin_epoch', default=1, type=int, help='Training begins at this epoch. Previous trained model indicated by resume_path is loaded.')
    parser.add_argument('--resume_path', default='', type=str, help='Save data (.pth) of previous training')
    parser.add_argument('--pretrain_path', default='Pretrained_EfficientFace.tar', type=str, help='Pretrained model (.pth), efficientface')
    parser.add_argument('--no_train', action='store_true', help='If true, training is not performed.')
    parser.set_defaults(no_train=False)
    parser.add_argument('--no_val', action='store_true', help='If true, validation is not performed.')
    parser.set_defaults(no_val=False)
    parser.add_argument('--test', action='store_true', help='If true, test is performed.')
    parser.set_defaults(test=True)
    parser.add_argument('--test_subset', default='test', type=str, help='Used subset in test (val | test)')
    
    parser.add_argument('--n_threads', default=1, type=int, help='Number of threads for multi-thread loading')
    parser.add_argument('--video_norm_value', default=255, type=int, help='If 1, range of inputs is [0-255]. If 255, range of inputs is [0-1].')
 
    parser.add_argument('--manual_seed', default=1, type=int, help='Manually set random seed')
    parser.add_argument('--fusion', default='ia', type=str, help='fusion type: lt | it | ia')
    parser.add_argument('--mask', type=str, help='dropout type : softhard | noise | nodropout', default='softhard')
    args = parser.parse_args([])

    return args

def val_ensemble(data_loader, model, criterion, opt, dist = None ):
    print("val_ensemble")
    #for evaluation with single modality, specify which modality to keep and which distortion to apply for the other modaltiy:
    #'noise', 'addnoise' or 'zeros'. for paper procedure, with 'softhard' mask use 'zeros' for evaluation, with 'noise' use 'noise' 
    model.eval()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    end_time = time.time()
    all_preds = []
    all_gts = []
    with torch.no_grad():
        for trial in range(100):
            preds =[]
            gts = []
            print(trial)
            for i, (inputs_audio, inputs_visual, targets) in enumerate(data_loader):
                data_time.update(time.time() - end_time)

                inputs_visual = inputs_visual.permute(0,2,1,3,4)
                inputs_visual = inputs_visual.reshape(inputs_visual.shape[0]*inputs_visual.shape[1], inputs_visual.shape[2], inputs_visual.shape[3], inputs_visual.shape[4])


                targets = targets.to(opt.device)
                with torch.no_grad():
                    inputs_visual = Variable(inputs_visual)
                    inputs_audio = Variable(inputs_audio)
                    targets = Variable(targets)
                outputs = model(inputs_audio.float(), inputs_visual.float())
                #print(torch.argmax(outputs,dim =1)[0],targets[0])
                preds.append(torch.argmax(outputs,dim =1)[0])
                gts.append(targets[0])
                loss = criterion(outputs, targets)
                prec1, prec5 = calculate_accuracy(outputs.data, targets.data, topk=(1,5))
                top1.update(prec1, inputs_audio.size(0))
                top5.update(prec5, inputs_audio.size(0))

                losses.update(loss.data, inputs_audio.size(0))

                batch_time.update(time.time() - end_time)
                end_time = time.time()
                
                print('Epoch: [{0}][{1}/{2}]\t'
                      'Time {batch_time.val:.5f} ({batch_time.avg:.5f})\t'
                      'Data {data_time.val:.5f} ({data_time.avg:.5f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Prec@1 {top1.val:.5f} ({top1.avg:.5f})\t'
                      'Prec@5 {top5.val:.5f} ({top5.avg:.5f})'.format(
                          0,
                          i + 1,
                          len(data_loader),
                          batch_time=batch_time,
                          data_time=data_time,
                          loss=losses,
                          top1=top1,
                          top5=top5))
               
            all_preds.append(torch.tensor(preds))
            all_gts.append(torch.tensor(gts))
    return all_preds, all_gts
if __name__ == '__main__':
    
    opt = parse_opts()
    n_folds = 1
    test_accuracies = []

    if opt.device != 'cpu':
        opt.device = 'cuda' if torch.cuda.is_available() else 'cpu' 
    if opt.certification_method == "MMCert":
        opt.result_path = "C:\\Users\\dongs\\Documents\\multimodal-emotion-recognition-main\\results\\MMCert_multimodalcnn_15_best0.pth"
    if opt.certification_method == "randomized_ablation":
        #opt.result_path = "C:\\Users\\dongs\\Documents\\multimodal-emotion-recognition-main\\results\\randomized_ablation_multimodalcnn_15_checkpoint0.pth"
        opt.result_path = "C:\\Users\\dongs\\Documents\\multimodal-emotion-recognition-main\\results\\randomized_ablation_multimodalcnn_15_best1.pth"
        #opt.result_path = "C:\\Users\\dongs\\Documents\\multimodal-emotion-recognition-main\\results\\randomized_ablation_multimodalcnn_15_best0.pth"
    pretrained = opt.pretrain_path != 'None'    
    
    #opt.result_path = 'res_'+str(time.time())
    if not os.path.exists(opt.result_path):
        os.makedirs(opt.result_path)
        
    opt.arch = '{}'.format(opt.model)  
    opt.store_name = '_'.join([opt.dataset, opt.model, str(opt.sample_duration)])
            

    #if opt.dataset == 'RAVDESS':
    #    opt.annotation_path = '/lustre/scratch/chumache/ravdess-develop/annotations_croppad_fold'+str(fold+1)+'.txt'

    print(opt)
    torch.manual_seed(opt.manual_seed)
    model, parameters = generate_model(opt)
    print("model_created")
    criterion = nn.CrossEntropyLoss()
    criterion = criterion.to(opt.device)

    video_transform = transforms.Compose([
        transforms.ToTensor(opt.video_norm_value)])

    test_data = get_test_set(opt, spatial_transform=video_transform) 
    print("dataset_created")
    #load best model
    best_state = torch.load(opt.result_path)
    model.load_state_dict(best_state['state_dict'])
    print("model_loaded")
    test_loader = torch.utils.data.DataLoader(
        test_data,
        batch_size=1,
        shuffle=False,
        num_workers=opt.n_threads,
        pin_memory=True)
    all_preds,all_gts = val_ensemble(test_loader, model, criterion, opt)

    print(all_preds)
    if opt.certification_method == "randomized_ablation":
        torch.save((all_preds,all_gts), 'output/'+opt.certification_method+"_k="+str(opt.k)+'_all_outputs.pth')
    if opt.certification_method == "MMCert":
        torch.save((all_preds,all_gts), 'output/'+opt.certification_method+"_k1="+str(opt.k1)+"_k2="+str(opt.k2)+'_all_outputs.pth')
    #print('Prec1: ' + str(test_prec1) + '; Loss: ' + str(test_loss))
    #test_accuracies.append(test_prec1) 
                


Namespace(phase='train', certification_method='MMCert', k1=4, k2=4000, k=3000, annotation_path='ravdess_preprocessing/annotations.txt', result_path='C:\\Users\\dongs\\Documents\\multimodal-emotion-recognition-main\\results\\MMCert_multimodalcnn_15_best0.pth', store_name='RAVDESS_multimodalcnn_15', dataset='RAVDESS', n_classes=8, model='multimodalcnn', num_heads=1, device='cuda', sample_size=224, sample_duration=15, learning_rate=0.04, momentum=0.9, lr_steps=[40, 55, 65, 70, 200, 250], dampening=0.9, weight_decay=0.001, lr_patience=10, batch_size=8, n_epochs=100, begin_epoch=1, resume_path='', pretrain_path='Pretrained_EfficientFace.tar', no_train=False, no_val=False, test=True, test_subset='test', n_threads=1, video_norm_value=255, manual_seed=1, fusion='ia', mask='softhard', arch='multimodalcnn')
Initializing efficientnet
Total number of trainable parameters:  1854766
model_created
dataset_created
model_loaded
val_ensemble
0
Epoch: [0][1/480]	Time 4.14883 (4.14883)	Data 4.13482 (4.134

Epoch: [0][52/480]	Time 0.09453 (0.17803)	Data 0.07952 (0.16237)	Loss 0.0009 (0.9003)	Prec@1 100.00000 (76.92308)	Prec@5 100.00000 (98.07693)
Epoch: [0][53/480]	Time 0.09753 (0.17651)	Data 0.08252 (0.16086)	Loss 5.0464 (0.9786)	Prec@1 0.00000 (75.47170)	Prec@5 100.00000 (98.11321)
Epoch: [0][54/480]	Time 0.10524 (0.17519)	Data 0.09023 (0.15955)	Loss 2.1137 (0.9996)	Prec@1 0.00000 (74.07407)	Prec@5 100.00000 (98.14815)
Epoch: [0][55/480]	Time 0.09953 (0.17381)	Data 0.08352 (0.15817)	Loss 3.8154 (1.0508)	Prec@1 0.00000 (72.72727)	Prec@5 100.00000 (98.18182)
Epoch: [0][56/480]	Time 0.10023 (0.17250)	Data 0.08622 (0.15689)	Loss 2.0572 (1.0687)	Prec@1 0.00000 (71.42857)	Prec@5 100.00000 (98.21429)
Epoch: [0][57/480]	Time 0.09952 (0.17122)	Data 0.08352 (0.15560)	Loss 0.3003 (1.0553)	Prec@1 100.00000 (71.92982)	Prec@5 100.00000 (98.24561)
Epoch: [0][58/480]	Time 0.10017 (0.16999)	Data 0.08316 (0.15435)	Loss 1.0586 (1.0553)	Prec@1 0.00000 (70.68965)	Prec@5 100.00000 (98.27586)
Epoch: [0][59/48

Epoch: [0][110/480]	Time 0.09151 (0.13396)	Data 0.07252 (0.11877)	Loss 0.0051 (1.0340)	Prec@1 100.00000 (73.63636)	Prec@5 100.00000 (98.18182)
Epoch: [0][111/480]	Time 0.08952 (0.13356)	Data 0.07452 (0.11837)	Loss 0.0256 (1.0249)	Prec@1 100.00000 (73.87388)	Prec@5 100.00000 (98.19820)
Epoch: [0][112/480]	Time 0.09153 (0.13319)	Data 0.07552 (0.11799)	Loss 0.0025 (1.0157)	Prec@1 100.00000 (74.10715)	Prec@5 100.00000 (98.21429)
Epoch: [0][113/480]	Time 0.09053 (0.13281)	Data 0.07452 (0.11761)	Loss 7.1895 (1.0704)	Prec@1 0.00000 (73.45132)	Prec@5 100.00000 (98.23009)
Epoch: [0][114/480]	Time 0.09553 (0.13248)	Data 0.07502 (0.11723)	Loss 1.9617 (1.0782)	Prec@1 0.00000 (72.80701)	Prec@5 100.00000 (98.24561)
Epoch: [0][115/480]	Time 0.08402 (0.13206)	Data 0.07102 (0.11683)	Loss 3.0058 (1.0950)	Prec@1 0.00000 (72.17391)	Prec@5 100.00000 (98.26086)
Epoch: [0][116/480]	Time 0.09262 (0.13172)	Data 0.07862 (0.11650)	Loss 2.1455 (1.1040)	Prec@1 0.00000 (71.55173)	Prec@5 100.00000 (98.27586)
Epoch: 

Epoch: [0][168/480]	Time 0.10253 (0.12090)	Data 0.08852 (0.10574)	Loss 0.0078 (0.9522)	Prec@1 100.00000 (74.40476)	Prec@5 100.00000 (98.80952)
Epoch: [0][169/480]	Time 0.09563 (0.12075)	Data 0.08088 (0.10560)	Loss 0.0049 (0.9466)	Prec@1 100.00000 (74.55621)	Prec@5 100.00000 (98.81657)
Epoch: [0][170/480]	Time 0.09353 (0.12059)	Data 0.07802 (0.10543)	Loss 0.0347 (0.9412)	Prec@1 100.00000 (74.70589)	Prec@5 100.00000 (98.82353)
Epoch: [0][171/480]	Time 0.09553 (0.12045)	Data 0.08202 (0.10530)	Loss 0.0077 (0.9357)	Prec@1 100.00000 (74.85381)	Prec@5 100.00000 (98.83041)
Epoch: [0][172/480]	Time 0.09653 (0.12031)	Data 0.08102 (0.10516)	Loss 0.0241 (0.9304)	Prec@1 100.00000 (75.00000)	Prec@5 100.00000 (98.83721)
Epoch: [0][173/480]	Time 0.09402 (0.12016)	Data 0.07702 (0.10499)	Loss 1.9801 (0.9365)	Prec@1 0.00000 (74.56647)	Prec@5 100.00000 (98.84393)
Epoch: [0][174/480]	Time 0.09629 (0.12002)	Data 0.08329 (0.10487)	Loss 0.0027 (0.9311)	Prec@1 100.00000 (74.71265)	Prec@5 100.00000 (98.85057)
E

Epoch: [0][228/480]	Time 0.08652 (0.11430)	Data 0.07352 (0.09934)	Loss 0.0006 (0.8115)	Prec@1 100.00000 (76.75439)	Prec@5 100.00000 (99.12281)
Epoch: [0][229/480]	Time 0.13404 (0.11439)	Data 0.11953 (0.09943)	Loss 0.0433 (0.8081)	Prec@1 100.00000 (76.85590)	Prec@5 100.00000 (99.12664)
Epoch: [0][230/480]	Time 0.09102 (0.11429)	Data 0.07702 (0.09933)	Loss 0.0074 (0.8046)	Prec@1 100.00000 (76.95652)	Prec@5 100.00000 (99.13043)
Epoch: [0][231/480]	Time 0.09304 (0.11420)	Data 0.07904 (0.09925)	Loss 0.0072 (0.8012)	Prec@1 100.00000 (77.05627)	Prec@5 100.00000 (99.13420)
Epoch: [0][232/480]	Time 0.09090 (0.11410)	Data 0.07590 (0.09915)	Loss 0.0105 (0.7978)	Prec@1 100.00000 (77.15517)	Prec@5 100.00000 (99.13793)
Epoch: [0][233/480]	Time 0.09253 (0.11400)	Data 0.07752 (0.09905)	Loss 1.9869 (0.8029)	Prec@1 0.00000 (76.82404)	Prec@5 100.00000 (99.14163)
Epoch: [0][234/480]	Time 0.09152 (0.11391)	Data 0.07752 (0.09896)	Loss 0.0055 (0.7995)	Prec@1 100.00000 (76.92308)	Prec@5 100.00000 (99.14530)
E

...

Epoch: [0][282/480]	Time 0.08552 (0.10230)	Data 0.06952 (0.08769)	Loss 0.6801 (0.9608)	Prec@1 100.00000 (71.25984)	Prec@5 100.00000 (99.47507)
Epoch: [0][283/480]	Time 0.08452 (0.10227)	Data 0.07152 (0.08766)	Loss 0.1787 (0.9598)	Prec@1 100.00000 (71.29752)	Prec@5 100.00000 (99.47575)
Epoch: [0][284/480]	Time 0.08952 (0.10226)	Data 0.07602 (0.08765)	Loss 0.0558 (0.9586)	Prec@1 100.00000 (71.33508)	Prec@5 100.00000 (99.47644)
Epoch: [0][285/480]	Time 0.08402 (0.10223)	Data 0.07102 (0.08763)	Loss 0.0002 (0.9573)	Prec@1 100.00000 (71.37255)	Prec@5 100.00000 (99.47713)
Epoch: [0][286/480]	Time 0.08440 (0.10221)	Data 0.06561 (0.08760)	Loss 0.0000 (0.9561)	Prec@1 100.00000 (71.40992)	Prec@5 100.00000 (99.47781)
Epoch: [0][287/480]	Time 0.08552 (0.10219)	Data 0.07152 (0.08758)	Loss 0.0000 (0.9548)	Prec@1 100.00000 (71.44720)	Prec@5 100.00000 (99.47849)
Epoch: [0][288/480]	Time 0.08252 (0.10216)	Data 0.06952 (0.08755)	Loss 0.0000 (0.9536)	Prec@1 100.00000 (71.48438)	Prec@5 100.00000 (99.47917)

Epoch: [0][342/480]	Time 0.08753 (0.10107)	Data 0.07352 (0.08649)	Loss 0.1746 (1.0087)	Prec@1 100.00000 (70.07299)	Prec@5 100.00000 (99.51338)
Epoch: [0][343/480]	Time 0.09052 (0.10106)	Data 0.07852 (0.08648)	Loss 2.2084 (1.0101)	Prec@1 0.00000 (69.98785)	Prec@5 100.00000 (99.51398)
Epoch: [0][344/480]	Time 0.08552 (0.10104)	Data 0.07252 (0.08647)	Loss 0.0356 (1.0089)	Prec@1 100.00000 (70.02427)	Prec@5 100.00000 (99.51456)
Epoch: [0][345/480]	Time 0.08752 (0.10103)	Data 0.07252 (0.08645)	Loss 0.0001 (1.0077)	Prec@1 100.00000 (70.06061)	Prec@5 100.00000 (99.51515)
Epoch: [0][346/480]	Time 0.08653 (0.10101)	Data 0.07353 (0.08644)	Loss 0.0000 (1.0065)	Prec@1 100.00000 (70.09686)	Prec@5 100.00000 (99.51574)
Epoch: [0][347/480]	Time 0.08352 (0.10099)	Data 0.06902 (0.08641)	Loss 0.0000 (1.0053)	Prec@1 100.00000 (70.13301)	Prec@5 100.00000 (99.51633)
Epoch: [0][348/480]	Time 0.08602 (0.10097)	Data 0.07302 (0.08640)	Loss 0.0000 (1.0041)	Prec@1 100.00000 (70.16908)	Prec@5 100.00000 (99.51691)
E

Epoch: [0][402/480]	Time 0.08952 (0.10003)	Data 0.07652 (0.08552)	Loss 0.7636 (0.9742)	Prec@1 0.00000 (70.63492)	Prec@5 100.00000 (99.54649)
Epoch: [0][403/480]	Time 0.08552 (0.10002)	Data 0.07102 (0.08551)	Loss 0.7579 (0.9740)	Prec@1 0.00000 (70.55492)	Prec@5 100.00000 (99.54700)
Epoch: [0][404/480]	Time 0.08602 (0.10000)	Data 0.07302 (0.08549)	Loss 0.0018 (0.9729)	Prec@1 100.00000 (70.58824)	Prec@5 100.00000 (99.54752)
Epoch: [0][405/480]	Time 0.08577 (0.09999)	Data 0.07152 (0.08548)	Loss 0.5481 (0.9724)	Prec@1 100.00000 (70.62147)	Prec@5 100.00000 (99.54803)
Epoch: [0][406/480]	Time 0.08554 (0.09997)	Data 0.07153 (0.08546)	Loss 0.1357 (0.9715)	Prec@1 100.00000 (70.65463)	Prec@5 100.00000 (99.54854)
Epoch: [0][407/480]	Time 0.08412 (0.09995)	Data 0.07111 (0.08544)	Loss 0.1036 (0.9705)	Prec@1 100.00000 (70.68771)	Prec@5 100.00000 (99.54903)
Epoch: [0][408/480]	Time 0.08452 (0.09993)	Data 0.07152 (0.08543)	Loss 0.0976 (0.9695)	Prec@1 100.00000 (70.72073)	Prec@5 100.00000 (99.54955)
Epo

Epoch: [0][462/480]	Time 0.08853 (0.09917)	Data 0.07402 (0.08475)	Loss 0.3966 (0.9460)	Prec@1 100.00000 (71.23142)	Prec@5 100.00000 (99.57538)
Epoch: [0][463/480]	Time 0.08502 (0.09916)	Data 0.07102 (0.08473)	Loss 0.9621 (0.9460)	Prec@1 0.00000 (71.15589)	Prec@5 100.00000 (99.57583)
Epoch: [0][464/480]	Time 0.08639 (0.09914)	Data 0.07239 (0.08472)	Loss 0.0471 (0.9450)	Prec@1 100.00000 (71.18644)	Prec@5 100.00000 (99.57627)
Epoch: [0][465/480]	Time 0.08753 (0.09913)	Data 0.07352 (0.08471)	Loss 0.0350 (0.9441)	Prec@1 100.00000 (71.21693)	Prec@5 100.00000 (99.57672)
Epoch: [0][466/480]	Time 0.09153 (0.09912)	Data 0.07452 (0.08470)	Loss 0.1816 (0.9433)	Prec@1 100.00000 (71.24735)	Prec@5 100.00000 (99.57716)
Epoch: [0][467/480]	Time 0.08152 (0.09911)	Data 0.06852 (0.08468)	Loss 2.4284 (0.9448)	Prec@1 0.00000 (71.17213)	Prec@5 100.00000 (99.57761)
Epoch: [0][468/480]	Time 0.08652 (0.09909)	Data 0.07352 (0.08467)	Loss 0.1996 (0.9440)	Prec@1 100.00000 (71.20253)	Prec@5 100.00000 (99.57806)
Epo

KeyboardInterrupt: 